In [1]:
import theano
import numpy as np
import pickle

In [2]:
def load_data():
    print 'Loading data....'
    with open('/Users/xingobar/Downloads/mnist.pkl', 'rb') as f:
        train_data,validation_data,test_data = pickle.load(f)
    
    def share_dataset(data,borrow=True):
        data_x,data_y = data
        shared_x = theano.shared(np.asarray(data_x,dtype=theano.config.floatX),borrow=borrow)
        shared_y = theano.shared(np.asarray(data_y,dtype=theano.config.floatX),borrow=borrow)
        return shared_x,shared_y
    
    train_x,train_y = share_dataset(train_data)
    validation_x,validation_y = share_dataset(validation_data)
    test_x,test_y = share_dataset(test_data)
    flatten = [(train_x,theano.tensor.cast(train_y.flatten(),'int32')),
               (validation_x,theano.tensor.cast(validation_y.flatten(),'int32')),
               (test_x,theano.tensor.cast(test_y.flatten(),'int32'))]
    return flatten

In [3]:
class LogisticRegressioin(object):
    def __init__(self,input,n_in,n_out):
        self.W = theano.shared(np.zeros((n_in,n_out),dtype=theano.config.floatX),name='weight',borrow=True)
        self.b = theano.shared(np.zeros((n_out),dtype=theano.config.floatX),name='bias',borrow=True)
        self.y_prob_given_x  =  theano.tensor.nnet.softmax(theano.tensor.dot(input,self.W) + self.b)
        self.y_pred = theano.tensor.argmax(self.y_prob_given_x,axis=1) 
        self.params = [self.W,self.b]
        self.input = input
    
    def negative_log_likelihood(self,y):
        return -theano.tensor.mean(theano.tensor.log(self.y_prob_given_x)[theano.tensor.arange(y.shape[0]),y])
    
    def error(self,y):
        if y.dtype.startswith('int'):
            return theano.tensor.mean(theano.tensor.neq(self.y_pred,y))

In [4]:
batch_size = 20
dataset = load_data()
train_x,train_y = dataset[0]
validation_x,validation_y = dataset[1]
n_train_batch = train_x.get_value(borrow=True).shape[0] / batch_size
n_validation_batch  = validation_x.get_value(borrow=True).shape[0] / batch_size 
print 'number of training batches : ' , n_train_batch

Loading data....
number of training batches :  2500


In [5]:
print 'building the model....'
index = theano.tensor.lscalar()
learning_rate = 0.01
x = theano.tensor.matrix('x')
y =  theano.tensor.ivector('y')
classifier = LogisticRegressioin(input = x , n_in = 28*28,n_out = 10)
cost  = classifier.negative_log_likelihood(y)

validation_model = theano.function(
inputs = [index],
outputs = classifier.error(y),
givens = {
        x:validation_x[index * batch_size : (index+1) * batch_size],
        y:validation_y[index * batch_size : (index+1) * batch_size]
    }
)
g_w = theano.tensor.grad(cost=cost,wrt=classifier.W)
g_b = theano.tensor.grad(cost=cost,wrt=classifier.b)

updates = [(classifier.W, classifier.W - learning_rate * g_w),
           (classifier.b,classifier.b - learning_rate * g_b)]

train_model = theano.function(
inputs = [index],
outputs = cost,
updates = updates,
givens = {
        x:train_x[index * batch_size : (index+1) * batch_size],
        y:train_y[index * batch_size : (index+1) * batch_size]
    }
)
print 'complete the training model'

building the model....
complete the training model
